## API Testing

In [1]:
import requests, json

In [ ]:


url = "https://fastapi-app-613987678533.asia-southeast1.run.app/upload-pill-image/"
files = {"file": open("test_pills/Bewell-C.jpg", "rb")}

response = requests.post(url, files=files)
response.json()

In [ ]:
import requests, json

url = "http://192.168.254.118:8000/upload-pill-image/"
files = {"file": open("test_pills/Bewell-C.jpg", "rb")}

response = requests.post(url, files=files)
response.json()

## Testing

In [2]:
import medication_matching
import ocr_module

[2025/06/17 12:45:56] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Jandrik/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Jandrik/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_

In [3]:
image_path = 'test_images/test2.jpg'

In [5]:
# Perform OCR on the downloaded image
ocr_text = ocr_module.extract_image_text(image_path=image_path)
# Get medication info
med_info = medication_matching.get_info(ocr_text, threshold=60)

[2025/06/17 12:47:05] ppocr DEBUG: dt_boxes num : 10, elapsed : 0.7110025882720947
[2025/06/17 12:47:05] ppocr DEBUG: cls num  : 10, elapsed : 0.13431549072265625
[2025/06/17 12:47:06] ppocr DEBUG: rec_res num  : 10, elapsed : 1.3132984638214111


In [1]:
from fuzzywuzzy import fuzz
import os

available_package_images_names = os.listdir('MedsForAll_Images')


In [ ]:
image_matches = []
threshold = 60

for match in med_info['matches']:
    match_name = f"{match['Generic Name']} {match['Brand Name']} {match['Dosage Strength']}"
    best_image_match = None
    best_match_score = 0
    for package_name in available_package_images_names:
        
        score = fuzz.token_sort_ratio(match_name, package_name[:-3].replace('-', '').strip())
        if score > best_match_score:
            best_match_score = score
            best_image_match = package_name

    if best_match_score >= threshold:
        image_matches.append(best_image_match)
    else:
        image_matches.append('')

In [2]:
import pickle

In [4]:
with open('pill_model/traintestsplit.pkl', 'rb') as tts_pkl:
    traintestsplit = pickle.load(tts_pkl)

In [7]:
pill_names_list = list(traintestsplit['class_to_label'].keys())

In [8]:
pill_names_list

['Aciclovir_Herpex_800_mg',
 'Ascorbic_Acid_BewellC',
 'Azithromycin_as_dihydrate_Zithromax',
 'Carbocisteine_Marluxyn',
 'Carbocisteine_Solmux',
 'DoloNeurobion_3B',
 'Iron__Pyridoxine__Cyanacobalamin__Folic_Acid_Hemarate',
 'KremilS',
 'Loperamide_Hydrochloride_Diatabs',
 'Loratadine_Allerta',
 'Loratadine_Claritin',
 'Losartan_Potassium_Medzart',
 'MX3',
 'Meclizine_Hydrochloride_Bonamine',
 'Mefenamic_Acid_Myrefen',
 'Montelukast_as_sodium__Levocetirizine_hydrochloride_Allerkast',
 'Multivatimins__Minerals_Centrum_Advance',
 'Naproxen_Sodium_Skelan_550',
 'Paracetamol_Biogesic',
 'Paracetamol_Tempaid',
 'Paracetamol_Tempra',
 'Phenylephrine_Hydrochloride__Chlorphenamine_Maleate__Paracetamol_Bioflu',
 'Rosuvastatin_Rosusaph10',
 'Sambong_Leaf_Uricare_500mg',
 'Sinecod_Forte',
 'Sodium_Ascorbate__Zinc_ImmunPro',
 'Vitex_Negundo_L_Lagundi_Leaf_Ascof_Forte',
 'Wild_Alaskan_Fish_Oil_1400_mg']